In [ ]:
server_name = ""
junk_table = ""

I've been waiting for this for 2 months, pulling data directly into python.  So of course I have no slides ready.

To pull data directly from SQL, we use the python database conenctor, pyodbc.  Since you all now have it (yay), let's start.

In [ ]:
import pyodbc

con = pyodbc.connect('DSN={}'.format(server_name))
query = "SELECT * FROM {}".format(junk_table)

print query
res = con.execute(query)
print res.fetchall()[:5]
con.close()

That code imports the python database connector library, establishes a connection to our Fox database, executes a simple query, prints its result, and closes the connection.

The line that won't work without some configuration is the connect line: I configure pyodbc to use my Windows credentials so that I don't have to store my password in python code.  You do this by clicking on the windows start button and typing "Data Sources (ODBC)" and opening that program once windows finds it.  

In the User DSN tab click Add... ==> select SQL Server Native Client 11.0 ==> Finish
On the next screen type the server's name [server for this lesson] in all 3 boxes (Name / Description / Server).
Click next until you see a finish button, and then "Test Data Source" if somehow this didn't work, google / ask a classmate after.

Try again after configuration, this time putting the connection object in a with block so that it closes once we're done with it:

In [ ]:
query = "SELECT * FROM {}".format(junk_table)

with pyodbc.connect('DSN={}'.format(server_name)) as con:
    res = con.execute(query)
    print res.fetchall()[:5]

Frequently, you want to save results to a file rather than just display them on your python terminal.  We can write a unicode csv easily with the following code.

In [ ]:
# ensure we have an output directory
import os
try:
    os.mkdir('output')
except WindowsError:
    pass

import unicodecsv
query = "SELECT * FROM {}".format(junk_table)

with pyodbc.connect('DSN={}'.format(server_name)) as con:
    res = con.execute(query)
    with open('output/out.csv', 'wb') as outf:
        writer = unicodecsv.writer(outf)
        # Write header information for each row
        writer.writerow([x[0] for x in res.description])
        for row in res:
            writer.writerow(row)
print "CSV file written"

df = pd.read_csv('output/out.csv')
print df.head()

You should now have a file "output/out.csv"  If you would rather have a more useful data format for the query results, like a pandas dataframe, or an excel file, we can do that too (!).

For excel files be careful that the file isn't open in excel when you try to have python write it: Excel locks the file for writing when it's open and is a giant pain.

In [ ]:
# ensure we have an output directory
import os
try:
    os.mkdir('output')
except WindowsError:
    pass


import pickle
import pandas as pd

query = "SELECT * FROM {}".format(junk_table)

with pyodbc.connect('DSN={}'.format(server_name)) as con:
    df = pd.read_sql(query, con)

df.to_excel('output/out.xlsx', 'query_sheet', index=False)
pickle.dump(df, open('output/out.df.pkl', 'wb'))

print df.head()

Not only can we write a single dataframe to a single excel sheet, but we can write multiple dataframes to the same excel sheet, all using pandas.

In [ ]:
writer = pd.ExcelWriter('output/multi_sheets.xlsx')
for i in range(5):
    temp = df.copy()
    if i != 0:
        temp[temp.columns[i]] = i
    temp.to_excel(writer, 'sheet{}'.format(i), index=False)
    del temp
writer.save()

print "Big excel file written"